In [1]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

In [2]:
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [3]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [5]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [7]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

2.199.0
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-141154829940/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-141154829940/boston-housing/input/validation/validation_dataset.csv


In [8]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('linear-learner', region)

print(container)

from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role() 

ll_estimator = Estimator(container,
    role=role, 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)

ll_estimator.set_hyperparameters(predictor_type='regressor', mini_batch_size=32)

382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [10]:
ll_estimator.fit(ll_data)

INFO:sagemaker:Creating training-job with name: linear-learner-2023-12-17-16-23-13-971


2023-12-17 16:23:14 Starting - Starting the training job...
2023-12-17 16:23:29 Starting - Preparing the instances for training......
2023-12-17 16:24:31 Downloading - Downloading input data......
2023-12-17 16:25:16 Downloading - Downloading the training image......
2023-12-17 16:26:42 Training - Training image download completed. Training in progress.
2023-12-17 16:26:42 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[12/17/2023 16:26:32 INFO 139702496220992] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 

In [11]:
%%bash -s "$ll_estimator.output_path"
aws s3 ls --recursive $1

2023-12-17 16:26:54          0 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/debug-output/training_job_end.ts
2023-12-17 16:26:53       1026 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/output/model.tar.gz
2023-12-17 16:26:54          0 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/profiler-output/framework/training_job_end.ts
2023-12-17 16:26:01      75656 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/profiler-output/system/incremental/2023121716/1702830240.algo-1.json
2023-12-17 16:26:00     184148 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/profiler-output/system/incremental/2023121716/1702830300.algo-1.json
2023-12-17 16:26:50     151804 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/profiler-output/system/incremental/2023121716/1702830360.algo-1.json
2023-12-17 16:26:54          0 boston-housing/output/linear-learner-2023-12-17-16-23-13-971/profiler-output/system/training_job_end.ts


In [12]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'linear-learner-demo-'+timestamp
print(endpoint_name)

linear-learner-demo-17-16-30-28


In [13]:
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: linear-learner-2023-12-17-16-30-46-965
INFO:sagemaker:Creating endpoint-config with name linear-learner-demo-17-16-30-28
INFO:sagemaker:Creating endpoint with name linear-learner-demo-17-16-30-28


--------------------!

In [14]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [15]:
#ll_predictor.content_type = 'text/csv'
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = ll_predictor.predict(test_sample)
print(response)

[['30.50813865661621']]


In [16]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = ll_predictor.predict(test_samples)
print(response)
print(ll_predictor.endpoint_name)

[['30.50813865661621'], ['25.279680252075195']]
linear-learner-demo-17-16-30-28


In [17]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


b'{"predictions": [{"score": 30.50813865661621}]}'


In [18]:
ll_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: linear-learner-demo-17-16-30-28
INFO:sagemaker:Deleting endpoint with name: linear-learner-demo-17-16-30-28
